In [0]:
import yaml

with open('../../configs/agent.yaml', 'r') as file:
    config = yaml.safe_load(file)

#load global configs
databricks_config = config['databricks_config']

#load uc configs
catalog=databricks_config['catalog']
schema=databricks_config['schema']
eval_table = databricks_config['eval_table_name']

In [0]:
import os
evals = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load(f"file://{os.getcwd()}/evals.tsv")
display(evals)

In [0]:
from pyspark.sql import functions as F

# starting df: inputs, expected_response
evals_final = (
    evals.select(
        # ── build the {"messages": [{"role":"user","content": <inputs>}] } struct ──
        F.struct(                                                            
            F.array(                                                         
                F.struct(                                                   
                    F.lit("user").alias("role"),                            
                    F.col("inputs").alias("content")                        
                )
            ).alias("messages")                                             
        ).alias("inputs"),                                                  

        # ── wrap expected_response ─────────────────────────────────────────
        F.struct(F.col("expected_response")).alias("expectations")
    )
)

In [0]:
display(evals_final)

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{eval_table}")

In [0]:
evals_final.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.{eval_table}")